In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression,Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetes-progression-b1/sample_submission.csv
/kaggle/input/diabetes-progression-b1/train.csv
/kaggle/input/diabetes-progression-b1/test.csv


In [2]:
test_data = pd.read_csv("/kaggle/input/diabetes-progression-b1/test.csv")
train_data = pd.read_csv("/kaggle/input/diabetes-progression-b1/train.csv")

In [3]:
X_train = train_data.drop('y', axis=1).drop("ID", axis=1)
y_train = train_data['y']
X_test = test_data.drop("ID", axis=1)

In [4]:
poly_features = MinMaxScaler()
p=PolynomialFeatures(degree=4)
x=p.fit_transform(X_train)
y=p.transform(X_test)
X_train_poly = poly_features.fit_transform(x)
X_test_poly = poly_features.transform(y)

In [5]:
reg_models = [
    ('linear', LinearRegression()),
    ('tree', DecisionTreeRegressor(min_samples_split=10)),
    ('forest', RandomForestRegressor(min_samples_split=10)),
    ('lasso', Lasso(max_iter=7900)),
    ('ridge', Ridge(max_iter=7900)),
    ('knn',KNeighborsRegressor(n_neighbors=5,weights='distance',p=3)),
]


In [6]:
ensemble_model = VotingRegressor(estimators=reg_models)

In [7]:
stratified_kfold = StratifiedKFold(n_splits = 5)  
cv_scores = cross_val_score(ensemble_model, X_train, y_train, cv=stratified_kfold, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-cv_scores)
ensemble_rmse = rmse_scores.mean()
print(f"Ensemble Model RMSE: {ensemble_rmse}")

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Ensemble Model RMSE: 55.26876388766474


In [8]:
ensemble_model.fit(X_train, y_train)

VotingRegressor(estimators=[('linear', LinearRegression()),
                            ('tree',
                             DecisionTreeRegressor(min_samples_split=10)),
                            ('forest',
                             RandomForestRegressor(min_samples_split=10)),
                            ('lasso', Lasso(max_iter=7900)),
                            ('ridge', Ridge(max_iter=7900)),
                            ('knn',
                             KNeighborsRegressor(p=3, weights='distance'))])

In [9]:
y_pred = ensemble_model.predict(X_test)

In [10]:
submission = pd.DataFrame({"ID": test_data["ID"], "y": y_pred})
submission.to_csv("submission.csv", index=False)